In [1]:
import numpy as np
import constriction

In [2]:
np.random.seed(202404151)
matrix = np.random.randn(1024, 1024)
matrix

array([[-0.24281341,  0.82277433, -1.76817447, ...,  0.37827248,
         0.95446806,  1.3724337 ],
       [-0.93972788,  0.53986172,  0.49766536, ...,  1.92457392,
        -0.32346853,  2.16728419],
       [ 0.77583578, -1.37499597, -0.03150158, ..., -1.95524463,
         0.77135577,  0.56992074],
       ...,
       [-1.11369009, -0.3860364 , -0.74349466, ...,  1.20332443,
         0.25760983,  1.11943256],
       [-0.73039511, -0.79896008, -0.80715964, ...,  0.54681539,
        -0.4218426 , -0.47858875],
       [ 0.11453367, -0.5499435 ,  0.71707023, ...,  0.45978248,
         0.60778436, -0.13992389]])

In [3]:
quantized_matrix = np.round(matrix * 8).astype(np.int8)
print(f'min: {quantized_matrix.min()}; max: {quantized_matrix.max()}')
quantized_matrix

min: -41; max: 38


array([[ -2,   7, -14, ...,   3,   8,  11],
       [ -8,   4,   4, ...,  15,  -3,  17],
       [  6, -11,   0, ..., -16,   6,   5],
       ...,
       [ -9,  -3,  -6, ...,  10,   2,   9],
       [ -6,  -6,  -6, ...,   4,  -3,  -4],
       [  1,  -4,   6, ...,   4,   5,  -1]], dtype=int8)

In [4]:
np.unique(quantized_matrix.ravel(), return_counts=True)

(array([-41, -38, -36, -35, -34, -33, -32, -31, -30, -29, -28, -27, -26,
        -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13,
        -12, -11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0,
          1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  38], dtype=int8),
 array([    1,     2,     3,     3,     6,     9,    20,    40,    36,
           73,   108,   187,   272,   409,   564,   862,  1159,  1656,
         2296,  3194,  4182,  5575,  7191,  9100, 11442, 13800, 17154,
        20278, 23604, 27970, 31746, 35848, 38960, 43094, 46095, 48689,
        50576, 51960, 52311, 51639, 50460, 48842, 46323, 43185, 39306,
        35732, 31837, 27474, 23987, 20475, 16945, 13971, 11278,  9067,
         7086,  5327,  4310,  3063,  2328,  1626,  1214,   853,   601,
          421,   254,   174,   122,    75,    54,    27,    2

In [171]:
values, counts = np.unique(quantized_matrix.ravel(), return_counts=True)
order = np.argsort(counts)[::-1]
values = values[order]
counts = counts[order]
entropy20bit = np.log2(matrix.size) - counts @ np.log2(counts) / matrix.size
entropy20bit

5.048731287507302

In [165]:
freqs = np.maximum(np.round(counts / 16).astype(np.int64), 1)
print(256**2 - sum(freqs), 1024**2 - sum(counts))
freqs[0] += 256**2 - sum(freqs) # very hacky but good enough for now
print(256**2 - sum(freqs), 1024**2 - sum(counts))
freqs

-8 0
0 0


array([3261, 3248, 3227, 3161, 3154, 3053, 3043, 2895, 2881, 2699, 2693,
       2457, 2435, 2240, 2233, 1990, 1984, 1748, 1717, 1499, 1475, 1280,
       1267, 1072, 1059,  873,  862,  715,  705,  569,  567,  449,  443,
        348,  333,  269,  261,  200,  191,  146,  144,  104,  102,   76,
         72,   54,   53,   38,   35,   26,   26,   17,   16,   12,   11,
          8,    7,    5,    5,    3,    2,    2,    2,    2,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1])

In [169]:
cross_entropy16bit = 16 - counts @ np.log2(freqs) / (1024**2)
cross_entropy16bit, entropy20bit

(5.048851110506442, 5.048731287507302)

In [172]:
cdf = np.add.accumulate(np.concatenate([np.array([0]), freqs]))
values_pad = np.concatenate([values, np.array([0])])
model_specification = np.stack((cdf, values_pad), axis=1).astype(np.int32).ravel()[:-1]
model_specification, len(model_specification), 2 * len(freqs), 1024**2

(array([    0,     0,  3261,    -1,  6509,     1,  9736,    -2, 12897,
            2, 16051,     3, 19104,    -3, 22147,     4, 25042,    -4,
        27923,     5, 30622,    -5, 33315,     6, 35772,    -6, 38207,
           -7, 40447,     7, 42680,     8, 44670,    -8, 46654,    -9,
        48402,     9, 50119,    10, 51618,   -10, 53093,    11, 54373,
          -11, 55640,   -12, 56712,    12, 57771,    13, 58644,   -13,
        59506,   -14, 60221,    14, 60926,   -15, 61495,    15, 62062,
          -16, 62511,    16, 62954,   -17, 63302,    17, 63635,    18,
        63904,   -18, 64165,   -19, 64365,    19, 64556,    20, 64702,
          -20, 64846,   -21, 64950,    21, 65052,    22, 65128,   -22,
        65200,   -23, 65254,    23, 65307,    24, 65345,   -24, 65380,
           25, 65406,   -25, 65432,   -26, 65449,    26, 65465,   -27,
        65477,    27, 65488,    28, 65496,   -28, 65503,    29, 65508,
          -29, 65513,    30, 65516,   -31, 65518,   -30, 65520,    31,
      

In [173]:
inv_vocab = {value: i for i, value in enumerate(values)}

In [174]:
distribution = constriction.stream.model.Categorical(freqs.astype(np.float64))
fake_next_data = np.array([1, 1], dtype=np.uint32) # pretend that there's more data to come (for the next matrix) so that decoding doesn't interfere with next row
def compress_row(row):
    row = np.array([inv_vocab[value] for value in row], dtype=np.int32)
    coder = constriction.stream.stack.AnsCoder(fake_next_data)
    coder.encode_reverse(row.astype(np.int32), distribution)
    return coder.get_compressed()

In [178]:
compressed = compress_row(quantized_matrix[0])
len(compressed) * 32 / 1024, cross_entropy16bit

(5.125, 5.048851110506442)

In [179]:
compressed = [compress_row(row) for row in quantized_matrix]

In [181]:
offsets = np.zeros((1024,), dtype=np.uint32)
pos = len(model_specification) + 1024
for i in range(1024):
    offsets[i] = pos
    pos += len(compressed[i])
offsets, len(offsets)

(array([  1177,   1341,   1503, ..., 167662, 167824, 167988], dtype=uint32),
 1024)

In [182]:
serialized = np.concatenate([offsets, model_specification.astype(np.uint32)] + compressed)

In [183]:
len(offsets) / 256

4.0

In [198]:
6509

6509

In [202]:
len(freqs), len(model_specification)

(76, 153)

In [200]:
model_specification

array([    0,     0,  3261,    -1,  6509,     1,  9736,    -2, 12897,
           2, 16051,     3, 19104,    -3, 22147,     4, 25042,    -4,
       27923,     5, 30622,    -5, 33315,     6, 35772,    -6, 38207,
          -7, 40447,     7, 42680,     8, 44670,    -8, 46654,    -9,
       48402,     9, 50119,    10, 51618,   -10, 53093,    11, 54373,
         -11, 55640,   -12, 56712,    12, 57771,    13, 58644,   -13,
       59506,   -14, 60221,    14, 60926,   -15, 61495,    15, 62062,
         -16, 62511,    16, 62954,   -17, 63302,    17, 63635,    18,
       63904,   -18, 64165,   -19, 64365,    19, 64556,    20, 64702,
         -20, 64846,   -21, 64950,    21, 65052,    22, 65128,   -22,
       65200,   -23, 65254,    23, 65307,    24, 65345,   -24, 65380,
          25, 65406,   -25, 65432,   -26, 65449,    26, 65465,   -27,
       65477,    27, 65488,    28, 65496,   -28, 65503,    29, 65508,
         -29, 65513,    30, 65516,   -31, 65518,   -30, 65520,    31,
       65522,    32,

In [189]:
# pad to a multiple of 256 entries for now so it's easier to read in
pad_len = (256 - len(serialized) % 256) % 256
serialized = np.concatenate([serialized, np.zeros((pad_len,), dtype=np.uint32)])

In [190]:
serialized.tofile('mock-matrix.bin')

In [127]:
serialized.shape, len(offsets), len(model_specification)

((168192,), 1024, 153)

In [193]:
!ls -l mock-matrix.bin

-rw-rw-r-- 1 robamler robamler 672768 Apr 16 15:16 mock-matrix.bin
